# LSTM model to do naam parser

In [1]:
!pip install fastprogress

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import string
import os

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer

## Data preprocessing

### Extract data from Florida dataset

In [4]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab/ethnicolor/data/fl_2022_FullName_train.csv.gz")
val_df = pd.read_csv("/content/drive/MyDrive/Colab/ethnicolor/data/fl_2022_FullName_val.csv.gz")
test_df = pd.read_csv("/content/drive/MyDrive/Colab/ethnicolor/data/fl_2022_FullName_test.csv.gz")

In [5]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(7214894, 9)
(901862, 9)
(901862, 9)


In [6]:
df = pd.concat([train_df, val_df, test_df])
drop_columns = [c for c in df.columns if c != 'full_name']
df = df.drop(drop_columns, axis=1)

In [7]:
df.shape

(9018618, 1)

In [8]:
df.head()

,full_name
0,Veit Karen
1,Barahona Octavio
2,Albert Jurieal
3,Foco Kevin
4,Ewen Deborah


In [9]:
df['last_name'], df['first_name'] = df['full_name'].str.split(' ', 1).str

<ipython-input-9-edeb4ca567a9>:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df['last_name'], df['first_name'] = df['full_name'].str.split(' ', 1).str
<ipython-input-9-edeb4ca567a9>:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['last_name'], df['first_name'] = df['full_name'].str.split(' ', 1).str


In [10]:
df_last_name = pd.DataFrame(df['last_name'].unique(), columns=['name'])
df_first_name = pd.DataFrame(df['first_name'].unique(), columns=['name'])

print(df_last_name.shape)
print(df_first_name.shape)

(739874, 1)
(969843, 1)


### Extract data from census

In [11]:
cn_train_df = pd.read_csv("/content/drive/MyDrive/Colab/ethnicolor/data/census_ln_train.csv")
cn_val_df = pd.read_csv("/content/drive/MyDrive/Colab/ethnicolor/data/census_ln_val.csv")
cn_test_df = pd.read_csv("/content/drive/MyDrive/Colab/ethnicolor/data/census_ln_test.csv")

In [12]:
cn_df = pd.concat([cn_train_df, cn_val_df, cn_test_df])
cn_df = cn_df.drop(['race','race_code'], axis=1)
cn_df = cn_df.rename(columns={'name_last': 'name'})
cn_df.head()

,name
0,Tamashiro
1,Pitkin
2,Gaydos
3,Ramaglia
4,Pampinella


In [13]:
df_last_name = pd.concat([df_last_name, cn_df])
df_last_name.drop_duplicates(inplace=True)
df_first_name.drop_duplicates(inplace=True)

print(df_last_name.shape)
print(df_first_name.shape)

(749852, 1)
(969843, 1)


In [14]:
df_last_name['name_type'] = 0
df_first_name['name_type'] = 1

final_df = pd.concat([df_last_name, df_first_name])
print(final_df.shape)

(1719695, 2)


## Split data to train, val and test

In [20]:
from sklearn.model_selection import train_test_split

train_df, rest_df = train_test_split(final_df, test_size=0.2, random_state=42, stratify=final_df['name_type'])
val_df, test_df = train_test_split(final_df, test_size=0.5, random_state=42, stratify=final_df['name_type'])

In [21]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(1375756, 2)
(859847, 2)
(859848, 2)


In [22]:
train_df.to_csv('/content/drive/MyDrive/Colab/ethnicolor/data/naamparser_train.csv', index=False)
val_df.to_csv('/content/drive/MyDrive/Colab/ethnicolor/data/naamparser_val.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/Colab/ethnicolor/data/naamparser_test.csv', index=False)